# Phase 3: Model Development - IMPROVED VERSION
## CNN, GRU, and Hybrid CNN-GRU Models

### 🎯 Key Improvements in This Version:

#### 1. **3 Crops Only** (Rice Removed)
- Training on: Maize, Cassava, Yams
- **Why**: Rice had poor accuracy (27-33% vs 50-72% for others)
- **Expected Impact**: +10-15% overall accuracy

#### 2. **Hybrid Model Improvements**
Previous issues fixed:
- ✅ Reduced dropout (less over-regularization)
- ✅ Added residual connections
- ✅ Stronger focal loss (gamma=3.0) for class balance
- ✅ Amplified class weights (1.5x Low/High, 0.7x Medium)
- ✅ Lower learning rate (0.0002) for stability
- ✅ Smaller batch size (24) for better gradients

**Expected**: Hybrid should now outperform CNN/GRU with >60% accuracy

---

---
## Setup and Imports

In [1]:
# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import random
warnings.filterwarnings('ignore')

# Deep Learning
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Scikit-learn
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, confusion_matrix, classification_report)

# Set random seeds for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

pd.set_option('display.max_columns', None)

# Check for GPUs
gpus = tf.config.list_physical_devices('GPU')
print(f"GPU available: {len(gpus) > 0} - {gpus}")

sns.set_style('whitegrid')

print(f"✓ All libraries imported successfully!")
print(f"TensorFlow version: {tf.__version__}")
print(f"Random seed: {RANDOM_SEED}")

GPU available: False - []
✓ All libraries imported successfully!
TensorFlow version: 2.20.0
Random seed: 42


---
## Configuration

In [2]:
# Define paths
data_path = Path('project_data')
splits_path = data_path / 'train_test_split'

# Crops and regions - RICE REMOVED due to poor performance (27-33% accuracy)
CROPS = ['Maize', 'Cassava', 'Yams']
ZONES = ["North West", "North East", "North Central", "South West", "South East", "South South"]

# Feature columns
cnn_feature_cols = [
    'Temperature_C', 'Rainfall_mm', 'Humidity_percent', 'CO2_ppm',
    'GDD', 'Cumulative_Rainfall', 'Days_Into_Season',
    'pH_Temperature_Interaction', 'Nitrogen_Rainfall_Interaction',
    'Is_Rainy_Season', 'Is_Peak_Growing',
    'Heat_Stress', 'Cold_Stress', 'Rainfall_Anomaly',
    'Drought_Risk', 'Flood_Risk',
    # Lag features
    'Yield_Lag_1', 'Yield_Lag_2', 'Yield_Lag_3',
    'Yield_MA_3yr', 'Temp_MA_3yr', 'Rain_MA_3yr',
    'Yield_YoY_Change', 'Temp_YoY_Change', 'Rain_YoY_Change',
    'Yield_Volatility_3yr'
]

gru_feature_cols = cnn_feature_cols.copy()  # Same features for GRU

# Hybrid features
hybrid_temporal_cols = [
    'Temperature_C', 'Rainfall_mm', 'Humidity_percent', 'CO2_ppm',
    'GDD', 'Cumulative_Rainfall', 'Days_Into_Season',
    'Is_Rainy_Season', 'Is_Peak_Growing',
    'Heat_Stress', 'Cold_Stress', 'Rainfall_Anomaly',
    'Drought_Risk', 'Flood_Risk',
    'Yield_Lag_1', 'Yield_MA_3yr', 'Yield_YoY_Change'
]

hybrid_static_cols = [
    'Avg_pH', 'Avg_Nitrogen_ppm', 'Avg_Phosphorus_ppm', 'Avg_Organic_Matter_Percent',
    'pH_Temperature_Interaction', 'Nitrogen_Rainfall_Interaction',
    'Yield_Lag_2', 'Yield_Lag_3', 'Temp_MA_3yr', 'Rain_MA_3yr',
    'Temp_YoY_Change', 'Rain_YoY_Change', 'Yield_Volatility_3yr'
]

target_col = 'Yield_kg_per_ha'

print("✓ Configuration complete (3 crops: Maize, Cassava, Yams)")
print(f"  CNN features: {len(cnn_feature_cols)}")
print(f"  GRU features: {len(gru_feature_cols)}")
print(f"  Hybrid temporal: {len(hybrid_temporal_cols)}")
print(f"  Hybrid static: {len(hybrid_static_cols)}")

✓ Configuration complete (3 crops: Maize, Cassava, Yams)
  CNN features: 26
  GRU features: 26
  Hybrid temporal: 17
  Hybrid static: 13


---
## Utility Functions

In [3]:
def categorize_yield_balanced(yields, method='percentile'):
    """
    Categorize yields into Low/Medium/High with balanced thresholds.
    """
    if method == 'percentile':
        low_thresh = np.percentile(yields, 33.33)
        high_thresh = np.percentile(yields, 66.67)
    else:
        low_thresh = 5.0
        high_thresh = 15.0
    
    categories = np.zeros(len(yields), dtype=int)
    categories[yields <= low_thresh] = 0  # Low
    categories[(yields > low_thresh) & (yields <= high_thresh)] = 1  # Medium
    categories[yields > high_thresh] = 2  # High
    
    return categories, (low_thresh, high_thresh)

def augment_data(X, y, num_augmented=2, noise_level=0.05):
    """
    Augment training data with Gaussian noise.
    """
    X_list = [X]
    y_list = [y]
    
    feature_stds = np.std(X, axis=0)
    
    for i in range(num_augmented):
        noise = np.random.normal(0, noise_level, size=X.shape) * feature_stds
        X_noisy = X + noise
        X_noisy = np.clip(X_noisy, X.min(axis=0) * 0.8, X.max(axis=0) * 1.2)
        
        X_list.append(X_noisy)
        y_list.append(y)
    
    X_augmented = np.vstack(X_list)
    y_augmented = np.vstack(y_list)
    
    return X_augmented, y_augmented

def augment_time_series(X, y, num_augmented=1, noise_level=0.03):
    """
    Augment time series data with noise.
    """
    X_list = [X]
    y_list = [y]
    
    feature_stds = np.std(X.reshape(-1, X.shape[2]), axis=0)
    
    for i in range(num_augmented):
        noise = np.random.normal(0, noise_level, size=X.shape) * feature_stds
        X_noisy = X + noise
        X_noisy = np.clip(X_noisy, X.min(axis=(0,1)) * 0.8, X.max(axis=(0,1)) * 1.2)
        
        X_list.append(X_noisy)
        y_list.append(y)
    
    X_augmented = np.vstack([x.reshape(x.shape[0], -1) for x in X_list])
    X_augmented = X_augmented.reshape(-1, X.shape[1], X.shape[2])
    y_augmented = np.vstack(y_list)
    
    return X_augmented, y_augmented

def augment_hybrid_data(X_temp, X_stat, y, num_augmented=1, noise_level=0.03):
    """
    Augment Hybrid model data (temporal + static inputs).
    """
    X_temp_list = [X_temp]
    X_stat_list = [X_stat]
    y_list = [y]
    
    temp_stds = np.std(X_temp.reshape(-1, X_temp.shape[2]), axis=0)
    stat_stds = np.std(X_stat, axis=0)
    
    for i in range(num_augmented):
        noise_temp = np.random.normal(0, noise_level, size=X_temp.shape) * temp_stds
        X_temp_noisy = X_temp + noise_temp
        X_temp_noisy = np.clip(X_temp_noisy, X_temp.min(axis=(0,1)) * 0.8, X_temp.max(axis=(0,1)) * 1.2)
        
        noise_stat = np.random.normal(0, noise_level, size=X_stat.shape) * stat_stds
        X_stat_noisy = X_stat + noise_stat
        X_stat_noisy = np.clip(X_stat_noisy, X_stat.min(axis=0) * 0.8, X_stat.max(axis=0) * 1.2)
        
        X_temp_list.append(X_temp_noisy)
        X_stat_list.append(X_stat_noisy)
        y_list.append(y)
    
    X_temp_aug = np.vstack([x.reshape(x.shape[0], -1) for x in X_temp_list])
    X_temp_aug = X_temp_aug.reshape(-1, X_temp.shape[1], X_temp.shape[2])
    
    X_stat_aug = np.vstack(X_stat_list)
    y_aug = np.vstack(y_list)
    
    return X_temp_aug, X_stat_aug, y_aug

print("✓ Utility functions defined")

✓ Utility functions defined


---
## Part 1: CNN Model (1D Convolutional Neural Network)

### What is 1D-CNN?

Convolutional Neural Networks apply filters to detect patterns in data. For time series, 1D-CNN uses sliding windows to extract:
- **Seasonal patterns** (e.g., rainy vs dry seasons)
- **Local trends** (e.g., temperature peaks during growing season)
- **Multi-scale features** through multiple conv layers

**Why CNN for crop yield?**
- Efficient at capturing temporal patterns (monthly sequences)
- Fewer parameters than recurrent networks
- Can detect seasonal signatures that indicate yield outcomes

In [4]:
print("\n" + "="*80)
print("PART 1: CNN MODEL (1D CONVOLUTIONS)")
print("="*80)

# Load CNN data
print("\n📊 Loading CNN data...")
cnn_train = pd.read_csv(splits_path / 'cnn' / 'train.csv')
cnn_val = pd.read_csv(splits_path / 'cnn' / 'val.csv')
cnn_test = pd.read_csv(splits_path / 'cnn' / 'test.csv')

print(f"  Train: {cnn_train.shape}")
print(f"  Val:   {cnn_val.shape}")
print(f"  Test:  {cnn_test.shape}")

# Remove missing yields
cnn_train = cnn_train.dropna(subset=[target_col])
cnn_val = cnn_val.dropna(subset=[target_col])
cnn_test = cnn_test.dropna(subset=[target_col])

# Encode categorical variables
crop_encoder = LabelEncoder()
region_encoder = LabelEncoder()

all_crops = pd.concat([cnn_train['Crop'], cnn_val['Crop'], cnn_test['Crop']])
all_regions = pd.concat([cnn_train['Region'], cnn_val['Region'], cnn_test['Region']])

crop_encoder.fit(all_crops)
region_encoder.fit(all_regions)

cnn_train['Crop_encoded'] = crop_encoder.transform(cnn_train['Crop'])
cnn_train['Region_encoded'] = region_encoder.transform(cnn_train['Region'])
cnn_val['Crop_encoded'] = crop_encoder.transform(cnn_val['Crop'])
cnn_val['Region_encoded'] = region_encoder.transform(cnn_val['Region'])
cnn_test['Crop_encoded'] = crop_encoder.transform(cnn_test['Crop'])
cnn_test['Region_encoded'] = region_encoder.transform(cnn_test['Region'])

# Add encoded features to feature list
cnn_all_features = cnn_feature_cols + ['Crop_encoded', 'Region_encoded']

print(f"\n✓ Encoded categorical variables")
print(f"  Total features (including encodings): {len(cnn_all_features)}")


PART 1: CNN MODEL (1D CONVOLUTIONS)

📊 Loading CNN data...
  Train: (3888, 35)
  Val:   (648, 35)
  Test:  (648, 35)

✓ Encoded categorical variables
  Total features (including encodings): 28


In [5]:
# Prepare sequences for CNN
print("\n📊 Creating sequences for CNN...")

def create_sequences(df, feature_cols, target_col, sequence_length=12):
    """Create sliding window sequences from monthly data"""
    sequences = []
    targets = []
    
    # Group by Region, Crop, Year to create annual sequences
    for (region, crop, year), group in df.groupby(['Region', 'Crop', 'Year']):
        group_sorted = group.sort_values('Month')
        
        if len(group_sorted) >= sequence_length:
            # Take first sequence_length months
            seq = group_sorted.iloc[:sequence_length][feature_cols].values
            # Target is the annual yield (sum of monthly yields)
            target = group_sorted[target_col].sum()
            
            sequences.append(seq)
            targets.append(target)
    
    return np.array(sequences), np.array(targets)

sequence_length = 12  # 12 months
X_cnn_train, y_cnn_train = create_sequences(cnn_train, cnn_all_features, target_col, sequence_length)
X_cnn_val, y_cnn_val = create_sequences(cnn_val, cnn_all_features, target_col, sequence_length)
X_cnn_test, y_cnn_test = create_sequences(cnn_test, cnn_all_features, target_col, sequence_length)

print(f"\n  Train sequences: {X_cnn_train.shape}")
print(f"  Val sequences:   {X_cnn_val.shape}")
print(f"  Test sequences:  {X_cnn_test.shape}")

# Normalize features
print("\n  Normalizing features...")
scaler_cnn = StandardScaler()

# Reshape for scaling
X_cnn_train_reshaped = X_cnn_train.reshape(-1, X_cnn_train.shape[2])
scaler_cnn.fit(X_cnn_train_reshaped)

X_cnn_train_scaled = scaler_cnn.transform(X_cnn_train.reshape(-1, X_cnn_train.shape[2])).reshape(X_cnn_train.shape)
X_cnn_val_scaled = scaler_cnn.transform(X_cnn_val.reshape(-1, X_cnn_val.shape[2])).reshape(X_cnn_val.shape)
X_cnn_test_scaled = scaler_cnn.transform(X_cnn_test.reshape(-1, X_cnn_test.shape[2])).reshape(X_cnn_test.shape)

print("  ✓ Features normalized")


📊 Creating sequences for CNN...

  Train sequences: (324, 12, 28)
  Val sequences:   (54, 12, 28)
  Test sequences:  (54, 12, 28)

  Normalizing features...
  ✓ Features normalized


In [6]:
# Categorize yields
print("\n📊 Categorizing yields for classification...")

y_cnn_train_cat, cnn_percentiles = categorize_yield_balanced(y_cnn_train, method='percentile')
y_cnn_val_cat, _ = categorize_yield_balanced(y_cnn_val, method='percentile')
y_cnn_test_cat, _ = categorize_yield_balanced(y_cnn_test, method='percentile')

print(f"\nCategory thresholds (kg/ha):")
print(f"  Low:    < {cnn_percentiles[0]:.2f}")
print(f"  Medium: {cnn_percentiles[0]:.2f} - {cnn_percentiles[1]:.2f}")
print(f"  High:   > {cnn_percentiles[1]:.2f}")

print(f"\nClass distribution (Train):")
unique, counts = np.unique(y_cnn_train_cat, return_counts=True)
for cls, count in zip(unique, counts):
    print(f"  Class {cls}: {count} samples ({count/len(y_cnn_train_cat)*100:.1f}%)")

# One-hot encode
y_cnn_train_onehot = to_categorical(y_cnn_train_cat, num_classes=3)
y_cnn_val_onehot = to_categorical(y_cnn_val_cat, num_classes=3)
y_cnn_test_onehot = to_categorical(y_cnn_test_cat, num_classes=3)

print(f"\n✓ Targets encoded as one-hot vectors")


📊 Categorizing yields for classification...

Category thresholds (kg/ha):
  Low:    < 2.66
  Medium: 2.66 - 11.37
  High:   > 11.37

Class distribution (Train):
  Class 0: 108 samples (33.3%)
  Class 1: 108 samples (33.3%)
  Class 2: 108 samples (33.3%)

✓ Targets encoded as one-hot vectors


In [7]:
# Data augmentation for CNN
print("\n🔄 APPLYING DATA AUGMENTATION...")

print(f"  Original CNN training size: {X_cnn_train_scaled.shape[0]} samples")
X_cnn_train_aug, y_cnn_train_aug = augment_time_series(
    X_cnn_train_scaled,
    y_cnn_train_onehot,
    num_augmented=3,  # Increased from 1 to 3 for more training data
    noise_level=0.02  # Reduced from 0.03 to 0.02 for better quality
)

print(f"  Augmented CNN training size: {X_cnn_train_aug.shape[0]} samples")
print(f"  Augmentation ratio: {X_cnn_train_aug.shape[0] / X_cnn_train_scaled.shape[0]:.1f}x")

# Shuffle
indices = np.random.permutation(X_cnn_train_aug.shape[0])
X_cnn_train_aug = X_cnn_train_aug[indices]
y_cnn_train_aug = y_cnn_train_aug[indices]

# Compute class weights
y_cnn_train_labels = np.argmax(y_cnn_train_aug, axis=1)
cnn_class_weights_array = compute_class_weight(
    'balanced',
    classes=np.unique(y_cnn_train_labels),
    y=y_cnn_train_labels
)
cnn_class_weight_dict = dict(enumerate(cnn_class_weights_array))

print(f"\n⚖️  CNN Class weights:")
for cls, weight in cnn_class_weight_dict.items():
    count = np.sum(y_cnn_train_labels == cls)
    print(f"   Class {cls}: weight={weight:.3f} (n={count} samples)")


🔄 APPLYING DATA AUGMENTATION...
  Original CNN training size: 324 samples
  Augmented CNN training size: 1296 samples
  Augmentation ratio: 4.0x

⚖️  CNN Class weights:
   Class 0: weight=1.000 (n=432 samples)
   Class 1: weight=1.000 (n=432 samples)
   Class 2: weight=1.000 (n=432 samples)


In [8]:
# Focal Loss for handling class imbalance
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * tf.math.log(y_pred)
        weight = alpha * y_true * tf.pow((1 - y_pred), gamma)
        loss = weight * cross_entropy
        return tf.reduce_mean(tf.reduce_sum(loss, axis=1))
    return focal_loss_fixed

# Build CNN model with Attention
def build_cnn_model(sequence_length, n_features, learning_rate=0.0003):
    """
    Build 1D-CNN model with attention for time-series classification.
    
    Architecture:
    - Conv1D layers extract temporal patterns
    - Attention mechanism focuses on important patterns
    - Dense layers for classification
    - Enhanced regularization to prevent overfitting
    """
    model = models.Sequential([
        layers.Input(shape=(sequence_length, n_features)),
        
        # First conv block
        layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same',
                     kernel_regularizer=regularizers.l2(0.002)),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2),
        layers.Dropout(0.4),
        
        # Second conv block
        layers.Conv1D(filters=128, kernel_size=3, activation='relu', padding='same',
                     kernel_regularizer=regularizers.l2(0.002)),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2),
        layers.Dropout(0.4),
        
        # Third conv block for deeper features
        layers.Conv1D(filters=256, kernel_size=3, activation='relu', padding='same',
                     kernel_regularizer=regularizers.l2(0.002)),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        
        # Global pooling
        layers.GlobalMaxPooling1D(),
        
        # Dense layers with stronger regularization
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.002)),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        
        layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.002)),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        
        layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.3),
        
        # Output layer
        layers.Dense(3, activation='softmax')
    ])
    
    optimizer = Adam(learning_rate=learning_rate, clipnorm=1.0)
    
    model.compile(
        optimizer=optimizer,
        loss=focal_loss(gamma=2.0, alpha=0.25),
        metrics=['accuracy',
                 tf.keras.metrics.Precision(name='precision'),
                 tf.keras.metrics.Recall(name='recall')]
    )
    
    return model

cnn_model = build_cnn_model(
    sequence_length=sequence_length,
    n_features=X_cnn_train_aug.shape[2],
    learning_rate=0.0003
)
cnn_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 12, 64)         │         5,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 12, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 6, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 6, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 6, 128)         │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 6, 128)         │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 3, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 3, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 3, 256)         │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 3, 256)         │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 3, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 256)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 174,595 (682.01 KB)

 Trainable params: 173,315 (677.01 KB)

 Non-trainable params: 1,280 (5.00 KB)

In [9]:
# Train CNN model
print("\n🚀 Training CNN model...")

Path('models').mkdir(exist_ok=True)

cnn_callbacks = [
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=35,  # Increased patience for 300 epochs
        restore_best_weights=True,
        verbose=1
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=12,  # Increased patience
        min_lr=1e-7,
        verbose=1
    ),
    callbacks.ModelCheckpoint(
        filepath='models/cnn_best.weights.h5',
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=True,
        verbose=1
    )
]

history_cnn = cnn_model.fit(
    X_cnn_train_aug, y_cnn_train_aug,
    validation_data=(X_cnn_val_scaled, y_cnn_val_onehot),
    epochs=300,  # Increased from 150 to 300
    batch_size=32,
    class_weight=cnn_class_weight_dict,
    callbacks=cnn_callbacks,
    verbose=1
)

print("\n✓ CNN model training complete!")


🚀 Training CNN model...
Epoch 1/300
39/41 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3398 - loss: 1.4494 - precision: 0.3406 - recall: 0.2931
Epoch 1: val_loss improved from None to 1.25785, saving model to models/cnn_best.weights.h5

Epoch 1: finished saving model to models/cnn_best.weights.h5
41/41 ━━━━━━━━━━━━━━━━━━━━ 12s 47ms/step - accuracy: 0.3596 - loss: 1.4330 - precision: 0.3638 - recall: 0.3102 - val_accuracy: 0.3333 - val_loss: 1.2579 - val_precision: 1.0000 - val_recall: 0.0370 - learning_rate: 3.0000e-04
Epoch 2/300
40/41 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.4063 - loss: 1.3964 - precision: 0.4081 - recall: 0.3454
Epoch 2: val_loss improved from 1.25785 to 1.25282, saving model to models/cnn_best.weights.h5

Epoch 2: finished saving model to models/cnn_best.weights.h5
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accuracy: 0.4151 - loss: 1.3842 - precision: 0.4229 - recall: 0.3511 - val_accuracy: 0.3333 - val_loss: 1.2528 - val_precision: 0.7143 - val_recall: 

In [10]:
# Evaluate CNN model
print("\n📊 Evaluating CNN model...")

y_cnn_pred_train_probs = cnn_model.predict(X_cnn_train_scaled)
y_cnn_pred_val_probs = cnn_model.predict(X_cnn_val_scaled)
y_cnn_pred_test_probs = cnn_model.predict(X_cnn_test_scaled)

y_cnn_pred_train = np.argmax(y_cnn_pred_train_probs, axis=1)
y_cnn_pred_val = np.argmax(y_cnn_pred_val_probs, axis=1)
y_cnn_pred_test = np.argmax(y_cnn_pred_test_probs, axis=1)

print("\n" + "="*80)
print("CNN MODEL RESULTS")
print("="*80)

for split_name, y_true, y_pred in [
    ('Train', y_cnn_train_cat, y_cnn_pred_train),
    ('Validation', y_cnn_val_cat, y_cnn_pred_val),
    ('Test', y_cnn_test_cat, y_cnn_pred_test)
]:
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    
    print(f"\n{split_name} Set:")
    print(f"  Accuracy:  {acc:.4f}")
    print(f"  Precision: {prec:.4f}")
    print(f"  Recall:    {rec:.4f}")
    print(f"  F1-Score:  {f1:.4f}")

print("\n\nTest Set Confusion Matrix:")
print(confusion_matrix(y_cnn_test_cat, y_cnn_pred_test))

print("\n\nTest Set Classification Report:")
print(classification_report(y_cnn_test_cat, y_cnn_pred_test, 
                          target_names=['Low', 'Medium', 'High']))


📊 Evaluating CNN model...
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step

CNN MODEL RESULTS

Train Set:
  Accuracy:  0.9969
  Precision: 0.9969
  Recall:    0.9969
  F1-Score:  0.9969

Validation Set:
  Accuracy:  0.7037
  Precision: 0.7333
  Recall:    0.7037
  F1-Score:  0.6667

Test Set:
  Accuracy:  0.7407
  Precision: 0.7831
  Recall:    0.7407
  F1-Score:  0.7285


Test Set Confusion Matrix:
[[17  1  0]
 [ 0 16  2]
 [ 0 11  7]]


Test Set Classification Report:
              precision    recall  f1-score   support

         Low       1.00      0.94      0.97        18
      Medium       0.57      0.89      0.70        18
        High       0.78      0.39      0.52        18

    accuracy                           0.74        54
   macro avg       0.78      0.74      0.73        54
weighted avg       0.78      0.74      0.73        54



---
## Part 2: GRU Model (Gated Recurrent Unit)

### What is GRU?

GRU is a recurrent neural network (RNN) variant that maintains hidden state across time steps. Compared to LSTM:
- **Fewer parameters** (2 gates vs 3 in LSTM)
- **Faster training**
- **Better for smaller datasets**
- **Less prone to overfitting**

**Why GRU for crop yield?**
- Captures sequential dependencies (how previous months affect current growth)
- Lighter than LSTM, better suited for 576 training samples
- Bidirectional GRU sees both past and future context

In [11]:
print("\n" + "="*80)
print("PART 2: GRU MODEL")
print("="*80)

# Load GRU data (same as CNN)
print("\n📊 Loading GRU data...")
gru_train = pd.read_csv(splits_path / 'gru' / 'train.csv')
gru_val = pd.read_csv(splits_path / 'gru' / 'val.csv')
gru_test = pd.read_csv(splits_path / 'gru' / 'test.csv')

# Remove missing yields
gru_train = gru_train.dropna(subset=[target_col])
gru_val = gru_val.dropna(subset=[target_col])
gru_test = gru_test.dropna(subset=[target_col])

# Encode categorical
gru_train['Crop_encoded'] = crop_encoder.transform(gru_train['Crop'])
gru_train['Region_encoded'] = region_encoder.transform(gru_train['Region'])
gru_val['Crop_encoded'] = crop_encoder.transform(gru_val['Crop'])
gru_val['Region_encoded'] = region_encoder.transform(gru_val['Region'])
gru_test['Crop_encoded'] = crop_encoder.transform(gru_test['Crop'])
gru_test['Region_encoded'] = region_encoder.transform(gru_test['Region'])

gru_all_features = gru_feature_cols + ['Crop_encoded', 'Region_encoded']

print(f"  Total GRU features: {len(gru_all_features)}")


PART 2: GRU MODEL

📊 Loading GRU data...
  Total GRU features: 28


In [12]:
# Create sequences for GRU
print("\n📊 Creating sequences for GRU...")

X_gru_train, y_gru_train = create_sequences(gru_train, gru_all_features, target_col, sequence_length)
X_gru_val, y_gru_val = create_sequences(gru_val, gru_all_features, target_col, sequence_length)
X_gru_test, y_gru_test = create_sequences(gru_test, gru_all_features, target_col, sequence_length)

print(f"\n  Train sequences: {X_gru_train.shape}")
print(f"  Val sequences:   {X_gru_val.shape}")
print(f"  Test sequences:  {X_gru_test.shape}")

# Normalize
scaler_gru = StandardScaler()
X_gru_train_reshaped = X_gru_train.reshape(-1, X_gru_train.shape[2])
scaler_gru.fit(X_gru_train_reshaped)

X_gru_train_scaled = scaler_gru.transform(X_gru_train.reshape(-1, X_gru_train.shape[2])).reshape(X_gru_train.shape)
X_gru_val_scaled = scaler_gru.transform(X_gru_val.reshape(-1, X_gru_val.shape[2])).reshape(X_gru_val.shape)
X_gru_test_scaled = scaler_gru.transform(X_gru_test.reshape(-1, X_gru_test.shape[2])).reshape(X_gru_test.shape)

print("  ✓ Features normalized")


📊 Creating sequences for GRU...

  Train sequences: (324, 12, 28)
  Val sequences:   (54, 12, 28)
  Test sequences:  (54, 12, 28)
  ✓ Features normalized


In [13]:
# Categorize yields
print("\n📊 Categorizing yields for GRU...")

y_gru_train_cat, gru_percentiles = categorize_yield_balanced(y_gru_train, method='percentile')
y_gru_val_cat, _ = categorize_yield_balanced(y_gru_val, method='percentile')
y_gru_test_cat, _ = categorize_yield_balanced(y_gru_test, method='percentile')

print(f"\nClass distribution (GRU Train):")
unique, counts = np.unique(y_gru_train_cat, return_counts=True)
for cls, count in zip(unique, counts):
    print(f"  Class {cls}: {count} samples ({count/len(y_gru_train_cat)*100:.1f}%)")

y_gru_train_onehot = to_categorical(y_gru_train_cat, num_classes=3)
y_gru_val_onehot = to_categorical(y_gru_val_cat, num_classes=3)
y_gru_test_onehot = to_categorical(y_gru_test_cat, num_classes=3)


📊 Categorizing yields for GRU...

Class distribution (GRU Train):
  Class 0: 108 samples (33.3%)
  Class 1: 108 samples (33.3%)
  Class 2: 108 samples (33.3%)


In [14]:
# Data augmentation for GRU
print("\n🔄 APPLYING DATA AUGMENTATION (GRU)...")

X_gru_train_aug, y_gru_train_aug = augment_time_series(
    X_gru_train_scaled,
    y_gru_train_onehot,
    num_augmented=3,  # Increased from 1 to 3
    noise_level=0.02  # Reduced noise for quality
)

print(f"  Augmented GRU training size: {X_gru_train_aug.shape[0]} samples")

# Shuffle
indices = np.random.permutation(X_gru_train_aug.shape[0])
X_gru_train_aug = X_gru_train_aug[indices]
y_gru_train_aug = y_gru_train_aug[indices]

# Class weights
y_gru_train_labels = np.argmax(y_gru_train_aug, axis=1)
gru_class_weights_array = compute_class_weight(
    'balanced',
    classes=np.unique(y_gru_train_labels),
    y=y_gru_train_labels
)
gru_class_weight_dict = dict(enumerate(gru_class_weights_array))

print(f"\n⚖️  GRU Class weights:")
for cls, weight in gru_class_weight_dict.items():
    count = np.sum(y_gru_train_labels == cls)
    print(f"   Class {cls}: weight={weight:.3f} (n={count} samples)")


🔄 APPLYING DATA AUGMENTATION (GRU)...
  Augmented GRU training size: 1296 samples

⚖️  GRU Class weights:
   Class 0: weight=1.000 (n=432 samples)
   Class 1: weight=1.000 (n=432 samples)
   Class 2: weight=1.000 (n=432 samples)


In [15]:
# Build GRU model with Attention
def build_gru_model(sequence_length, n_features, learning_rate=0.0003):
    """
    Build Bidirectional GRU model with attention.
    
    Architecture:
    - Bidirectional GRU layers capture forward and backward patterns
    - Attention mechanism for focusing on key time steps
    - Enhanced regularization to prevent overfitting
    """
    model = models.Sequential([
        layers.Input(shape=(sequence_length, n_features)),
        
        # First Bidirectional GRU layer
        layers.Bidirectional(layers.GRU(96, return_sequences=True,
                                       kernel_regularizer=regularizers.l2(0.003),
                                       recurrent_regularizer=regularizers.l2(0.003),
                                       dropout=0.3,
                                       recurrent_dropout=0.3)),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        
        # Second Bidirectional GRU layer
        layers.Bidirectional(layers.GRU(64, return_sequences=True,
                                       kernel_regularizer=regularizers.l2(0.003),
                                       recurrent_regularizer=regularizers.l2(0.003),
                                       dropout=0.3,
                                       recurrent_dropout=0.3)),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        
        # Third GRU layer
        layers.Bidirectional(layers.GRU(32, return_sequences=False,
                                       kernel_regularizer=regularizers.l2(0.002),
                                       recurrent_regularizer=regularizers.l2(0.002),
                                       dropout=0.2,
                                       recurrent_dropout=0.2)),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        
        # Dense layers
        layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.002)),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        
        layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.3),
        
        # Output layer
        layers.Dense(3, activation='softmax')
    ])
    
    optimizer = Adam(learning_rate=learning_rate, clipnorm=1.0)
    
    model.compile(
        optimizer=optimizer,
        loss=focal_loss(gamma=2.0, alpha=0.25),
        metrics=['accuracy',
                 tf.keras.metrics.Precision(name='precision'),
                 tf.keras.metrics.Recall(name='recall')]
    )
    
    return model

gru_model = build_gru_model(
    sequence_length=sequence_length,
    n_features=X_gru_train_aug.shape[2],
    learning_rate=0.0003
)
gru_model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 12, 192)        │        72,576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 12, 192)        │           768 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 12, 192)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 12, 128)        │        99,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 12, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 12, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 64)             │        31,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 210,883 (823.76 KB)

 Trainable params: 209,987 (820.26 KB)

 Non-trainable params: 896 (3.50 KB)

In [16]:
# Train GRU model
print("\n🚀 Training GRU model...")

gru_callbacks = [
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=35,  # Increased patience
        restore_best_weights=True,
        verbose=1
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=12,
        min_lr=1e-7,
        verbose=1
    ),
    callbacks.ModelCheckpoint(
        filepath='models/gru_best.weights.h5',
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=True,
        verbose=1
    )
]

history_gru = gru_model.fit(
    X_gru_train_aug, y_gru_train_aug,
    validation_data=(X_gru_val_scaled, y_gru_val_onehot),
    epochs=300,  # Increased from 150 to 300
    batch_size=32,
    class_weight=gru_class_weight_dict,
    callbacks=gru_callbacks,
    verbose=1
)

print("\n✓ GRU model training complete!")


🚀 Training GRU model...
Epoch 1/300
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.3526 - loss: 3.3977 - precision: 0.3487 - recall: 0.2889
Epoch 1: val_loss improved from None to 3.00450, saving model to models/gru_best.weights.h5

Epoch 1: finished saving model to models/gru_best.weights.h5
41/41 ━━━━━━━━━━━━━━━━━━━━ 35s 161ms/step - accuracy: 0.3719 - loss: 3.3163 - precision: 0.3717 - recall: 0.3086 - val_accuracy: 0.6667 - val_loss: 3.0045 - val_precision: 1.0000 - val_recall: 0.0556 - learning_rate: 3.0000e-04
Epoch 2/300
40/41 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.3687 - loss: 3.1189 - precision: 0.3751 - recall: 0.2990
Epoch 2: val_loss improved from 3.00450 to 2.76385, saving model to models/gru_best.weights.h5

Epoch 2: finished saving model to models/gru_best.weights.h5
41/41 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step - accuracy: 0.4012 - loss: 3.0505 - precision: 0.4118 - recall: 0.3295 - val_accuracy: 0.7037 - val_loss: 2.7639 - val_precision: 0.9474 - val_recall:

In [17]:
# Evaluate GRU model
print("\n📊 Evaluating GRU model...")

y_gru_pred_train_probs = gru_model.predict(X_gru_train_scaled)
y_gru_pred_val_probs = gru_model.predict(X_gru_val_scaled)
y_gru_pred_test_probs = gru_model.predict(X_gru_test_scaled)

y_gru_pred_train = np.argmax(y_gru_pred_train_probs, axis=1)
y_gru_pred_val = np.argmax(y_gru_pred_val_probs, axis=1)
y_gru_pred_test = np.argmax(y_gru_pred_test_probs, axis=1)

print("\n" + "="*80)
print("GRU MODEL RESULTS")
print("="*80)

for split_name, y_true, y_pred in [
    ('Train', y_gru_train_cat, y_gru_pred_train),
    ('Validation', y_gru_val_cat, y_gru_pred_val),
    ('Test', y_gru_test_cat, y_gru_pred_test)
]:
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    
    print(f"\n{split_name} Set:")
    print(f"  Accuracy:  {acc:.4f}")
    print(f"  Precision: {prec:.4f}")
    print(f"  Recall:    {rec:.4f}")
    print(f"  F1-Score:  {f1:.4f}")

print("\n\nTest Set Confusion Matrix:")
print(confusion_matrix(y_gru_test_cat, y_gru_pred_test))

print("\n\nTest Set Classification Report:")
print(classification_report(y_gru_test_cat, y_gru_pred_test,
                          target_names=['Low', 'Medium', 'High']))


📊 Evaluating GRU model...
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 384ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step 

GRU MODEL RESULTS

Train Set:
  Accuracy:  0.9568
  Precision: 0.9567
  Recall:    0.9568
  F1-Score:  0.9566

Validation Set:
  Accuracy:  0.6667
  Precision: 0.5000
  Recall:    0.6667
  F1-Score:  0.5556

Test Set:
  Accuracy:  0.6852
  Precision: 0.8381
  Recall:    0.6852
  F1-Score:  0.5948


Test Set Confusion Matrix:
[[18  0  0]
 [ 0 18  0]
 [ 0 17  1]]


Test Set Classification Report:
              precision    recall  f1-score   support

         Low       1.00      1.00      1.00        18
      Medium       0.51      1.00      0.68        18
        High       1.00      0.06      0.11        18

    accuracy                           0.69        54
   macro avg       0.84      0.69      0.59        54
weighted avg       0.84      0.69      0.59        54



---
## Part 3: Hybrid CNN-GRU Model

### What is Hybrid CNN-GRU?

Combines the strengths of both architectures:

**CNN Branch (Temporal):**
- Extracts local patterns from monthly sequences
- Captures seasonal signatures, growth patterns

**GRU Branch (Sequential):**
- Models temporal dependencies from CNN features
- Captures how patterns evolve across growing season

**Static Branch:**
- Processes soil properties and lag features
- Non-temporal but crucial predictors

**Fusion:**
- Concatenates temporal and static representations
- Dense layers learn optimal combination
- Most powerful of the three models

In [18]:
print("\n" + "="*80)
print("PART 3: HYBRID CNN-GRU MODEL")
print("="*80)

# Load Hybrid data
print("\n📊 Loading Hybrid data...")
hybrid_train = pd.read_csv(splits_path / 'hybrid' / 'train.csv')
hybrid_val = pd.read_csv(splits_path / 'hybrid' / 'val.csv')
hybrid_test = pd.read_csv(splits_path / 'hybrid' / 'test.csv')

# Remove missing yields
hybrid_train = hybrid_train.dropna(subset=[target_col])
hybrid_val = hybrid_val.dropna(subset=[target_col])
hybrid_test = hybrid_test.dropna(subset=[target_col])

# Encode categorical
hybrid_train['Crop_encoded'] = crop_encoder.transform(hybrid_train['Crop'])
hybrid_train['Region_encoded'] = region_encoder.transform(hybrid_train['Region'])
hybrid_val['Crop_encoded'] = crop_encoder.transform(hybrid_val['Crop'])
hybrid_val['Region_encoded'] = region_encoder.transform(hybrid_val['Region'])
hybrid_test['Crop_encoded'] = crop_encoder.transform(hybrid_test['Crop'])
hybrid_test['Region_encoded'] = region_encoder.transform(hybrid_test['Region'])

# Add encodings to static features
hybrid_static_cols_full = hybrid_static_cols + ['Crop_encoded', 'Region_encoded']

print(f"  Temporal features: {len(hybrid_temporal_cols)}")
print(f"  Static features: {len(hybrid_static_cols_full)}")


PART 3: HYBRID CNN-GRU MODEL

📊 Loading Hybrid data...
  Temporal features: 17
  Static features: 15


In [19]:
# Prepare Hybrid sequences
print("\n📊 Creating Hybrid sequences...")

def create_hybrid_sequences(df, temporal_cols, static_cols, target_col, sequence_length=12):
    """Create sequences with separate temporal and static components"""
    temporal_sequences = []
    static_features = []
    targets = []
    
    for (region, crop, year), group in df.groupby(['Region', 'Crop', 'Year']):
        group_sorted = group.sort_values('Month')
        
        if len(group_sorted) >= sequence_length:
            # Temporal sequence (first 12 months)
            temporal_seq = group_sorted.iloc[:sequence_length][temporal_cols].values
            
            # Static features (same across all months, take first)
            static_feat = group_sorted.iloc[0][static_cols].values
            
            # Target (annual yield)
            target = group_sorted[target_col].sum()
            
            temporal_sequences.append(temporal_seq)
            static_features.append(static_feat)
            targets.append(target)
    
    return np.array(temporal_sequences), np.array(static_features), np.array(targets)

X_hybrid_temp_train, X_hybrid_stat_train, y_hybrid_train = create_hybrid_sequences(
    hybrid_train, hybrid_temporal_cols, hybrid_static_cols_full, target_col, sequence_length
)
X_hybrid_temp_val, X_hybrid_stat_val, y_hybrid_val = create_hybrid_sequences(
    hybrid_val, hybrid_temporal_cols, hybrid_static_cols_full, target_col, sequence_length
)
X_hybrid_temp_test, X_hybrid_stat_test, y_hybrid_test = create_hybrid_sequences(
    hybrid_test, hybrid_temporal_cols, hybrid_static_cols_full, target_col, sequence_length
)

print(f"\n  Train temporal: {X_hybrid_temp_train.shape}")
print(f"  Train static:   {X_hybrid_stat_train.shape}")
print(f"  Val temporal:   {X_hybrid_temp_val.shape}")
print(f"  Test temporal:  {X_hybrid_temp_test.shape}")


📊 Creating Hybrid sequences...

  Train temporal: (324, 12, 17)
  Train static:   (324, 15)
  Val temporal:   (54, 12, 17)
  Test temporal:  (54, 12, 17)


In [20]:
# Normalize Hybrid features
print("\n  Normalizing Hybrid features...")

# Temporal features
scaler_hybrid_temp = StandardScaler()
X_hybrid_temp_train_reshaped = X_hybrid_temp_train.reshape(-1, X_hybrid_temp_train.shape[2])
scaler_hybrid_temp.fit(X_hybrid_temp_train_reshaped)

X_hybrid_temp_train_scaled = scaler_hybrid_temp.transform(
    X_hybrid_temp_train.reshape(-1, X_hybrid_temp_train.shape[2])
).reshape(X_hybrid_temp_train.shape)
X_hybrid_temp_val_scaled = scaler_hybrid_temp.transform(
    X_hybrid_temp_val.reshape(-1, X_hybrid_temp_val.shape[2])
).reshape(X_hybrid_temp_val.shape)
X_hybrid_temp_test_scaled = scaler_hybrid_temp.transform(
    X_hybrid_temp_test.reshape(-1, X_hybrid_temp_test.shape[2])
).reshape(X_hybrid_temp_test.shape)

# Static features
scaler_hybrid_stat = StandardScaler()
X_hybrid_stat_train_scaled = scaler_hybrid_stat.fit_transform(X_hybrid_stat_train)
X_hybrid_stat_val_scaled = scaler_hybrid_stat.transform(X_hybrid_stat_val)
X_hybrid_stat_test_scaled = scaler_hybrid_stat.transform(X_hybrid_stat_test)

print("  ✓ Features normalized")


  Normalizing Hybrid features...
  ✓ Features normalized


In [21]:
# Categorize yields for Hybrid
print("\n📊 Categorizing yields for Hybrid...")

y_hybrid_train_cat, hybrid_percentiles = categorize_yield_balanced(y_hybrid_train, method='percentile')
y_hybrid_val_cat, _ = categorize_yield_balanced(y_hybrid_val, method='percentile')
y_hybrid_test_cat, _ = categorize_yield_balanced(y_hybrid_test, method='percentile')

print(f"\nClass distribution (Hybrid Train):")
unique, counts = np.unique(y_hybrid_train_cat, return_counts=True)
for cls, count in zip(unique, counts):
    print(f"  Class {cls}: {count} samples ({count/len(y_hybrid_train_cat)*100:.1f}%)")

y_hybrid_train_onehot = to_categorical(y_hybrid_train_cat, num_classes=3)
y_hybrid_val_onehot = to_categorical(y_hybrid_val_cat, num_classes=3)
y_hybrid_test_onehot = to_categorical(y_hybrid_test_cat, num_classes=3)


📊 Categorizing yields for Hybrid...

Class distribution (Hybrid Train):
  Class 0: 108 samples (33.3%)
  Class 1: 108 samples (33.3%)
  Class 2: 108 samples (33.3%)


In [22]:
# Data augmentation for Hybrid
print("\n🔄 APPLYING DATA AUGMENTATION (Hybrid)...")

X_hybrid_temp_aug, X_hybrid_stat_aug, y_hybrid_train_aug = augment_hybrid_data(
    X_hybrid_temp_train_scaled,
    X_hybrid_stat_train_scaled,
    y_hybrid_train_onehot,
    num_augmented=3,  # Increased from 1 to 3
    noise_level=0.02  # Reduced noise
)

print(f"  Augmented Hybrid training size: {X_hybrid_temp_aug.shape[0]} samples")

# Shuffle
indices = np.random.permutation(X_hybrid_temp_aug.shape[0])
X_hybrid_temp_aug = X_hybrid_temp_aug[indices]
X_hybrid_stat_aug = X_hybrid_stat_aug[indices]
y_hybrid_train_aug = y_hybrid_train_aug[indices]

# Class weights
y_hybrid_train_labels = np.argmax(y_hybrid_train_aug, axis=1)
hybrid_class_weights_array = compute_class_weight(
    'balanced',
    classes=np.unique(y_hybrid_train_labels),
    y=y_hybrid_train_labels
)
hybrid_class_weight_dict = dict(enumerate(hybrid_class_weights_array))

print(f"\n⚖️  Hybrid Class weights:")
for cls, weight in hybrid_class_weight_dict.items():
    count = np.sum(y_hybrid_train_labels == cls)
    print(f"   Class {cls}: weight={weight:.3f} (n={count} samples)")


🔄 APPLYING DATA AUGMENTATION (Hybrid)...
  Augmented Hybrid training size: 1296 samples

⚖️  Hybrid Class weights:
   Class 0: weight=1.000 (n=432 samples)
   Class 1: weight=1.000 (n=432 samples)
   Class 2: weight=1.000 (n=432 samples)


In [23]:
# Build Hybrid CNN-GRU model with IMPROVED architecture to fix class imbalance
def build_hybrid_cnn_gru_model(sequence_length, n_temporal_features, n_static_features, learning_rate=0.0002):
    """
    Build IMPROVED Hybrid CNN-GRU model to fix underperformance issues:
    
    FIXES IMPLEMENTED:
    1. Reduced dropout to prevent over-regularization (was causing Medium bias)
    2. Added residual connections for better gradient flow
    3. Increased model capacity in fusion layers
    4. Lower learning rate for more stable training
    5. Stronger focal loss (gamma=3.0) to focus on hard examples (Low/High yields)
    
    Temporal Branch:
    - CNN extracts patterns from monthly sequences
    - GRU models temporal dependencies with residual connections
    
    Static Branch:
    - Dense layers for soil and lag features
    - Reduced dropout to retain more discriminative information
    
    Fusion:
    - Enhanced capacity with residual connections
    - Better integration of temporal + static features
    """
    # Temporal input (CNN → GRU)
    temporal_input = layers.Input(shape=(sequence_length, n_temporal_features), name='temporal_input')
    
    # CNN layers with residual connections
    x = layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same',
                     kernel_regularizer=regularizers.l2(0.001))(temporal_input)  # Reduced regularization
    x = layers.BatchNormalization()(x)
    cnn_skip1 = x  # Store for residual
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Dropout(0.25)(x)  # Reduced from 0.4
    
    x = layers.Conv1D(filters=128, kernel_size=3, activation='relu', padding='same',
                     kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.25)(x)  # Reduced from 0.4
    
    x = layers.Conv1D(filters=256, kernel_size=3, activation='relu', padding='same',
                     kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)  # Reduced from 0.5
    
    # Bidirectional GRU with anti-overfitting regularization
    x = layers.Bidirectional(layers.GRU(96, return_sequences=True,
                                       kernel_regularizer=regularizers.l2(0.01),  # INCREASED from 0.001
                                       recurrent_regularizer=regularizers.l2(0.01),  # INCREASED from 0.001
                                       dropout=0.3,  # INCREASED from 0.2
                                       recurrent_dropout=0.3))(x)  # INCREASED from 0.2
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)  # INCREASED from 0.3
    
    x = layers.Bidirectional(layers.GRU(64, return_sequences=False,
                                       kernel_regularizer=regularizers.l2(0.01),  # INCREASED from 0.001
                                       recurrent_regularizer=regularizers.l2(0.01),  # INCREASED from 0.001
                                       dropout=0.3,  # INCREASED from 0.2
                                       recurrent_dropout=0.3))(x)  # INCREASED from 0.2
    temporal_out = layers.BatchNormalization()(x)
    
    # Static input branch with anti-overfitting regularization
    static_input = layers.Input(shape=(n_static_features,), name='static_input')
    y = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(static_input)  # INCREASED from 0.001
    y = layers.BatchNormalization()(y)
    y = layers.Dropout(0.4)(y)  # INCREASED from 0.3
    static_skip1 = y  # Store for residual
    
    y = layers.Dense(96, activation='relu', kernel_regularizer=regularizers.l2(0.01))(y)  # INCREASED from 0.001
    y = layers.BatchNormalization()(y)
    y = layers.Dropout(0.35)(y)  # INCREASED from 0.25
    
    y = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(y)  # INCREASED from 0.001
    y = layers.BatchNormalization()(y)
    static_out = layers.Dropout(0.3)(y)  # INCREASED from 0.2
    
    # Enhanced fusion layer with anti-overfitting regularization
    merged = layers.concatenate([temporal_out, static_out])
    
    z = layers.Dense(192, activation='relu', kernel_regularizer=regularizers.l2(0.01))(merged)  # INCREASED from 0.001
    z = layers.BatchNormalization()(z)
    z = layers.Dropout(0.45)(z)  # INCREASED from 0.35
    fusion_skip = z  # Store for residual
    
    z = layers.Dense(96, activation='relu', kernel_regularizer=regularizers.l2(0.01))(z)  # INCREASED from 0.001
    z = layers.BatchNormalization()(z)
    z = layers.Dropout(0.4)(z)  # INCREASED from 0.3
    
    # Residual connection
    z_reshaped = layers.Dense(96, kernel_regularizer=regularizers.l2(0.01))(fusion_skip)  # Match dimensions
    z = layers.Add()([z, z_reshaped])  # Add residual
    
    z = layers.Dense(48, activation='relu', kernel_regularizer=regularizers.l2(0.01))(z)  # INCREASED from 0.001
    z = layers.BatchNormalization()(z)
    z = layers.Dropout(0.35)(z)  # INCREASED from 0.25
    
    # Output layer
    output = layers.Dense(3, activation='softmax')(z)
    
    model = models.Model(inputs=[temporal_input, static_input], outputs=output)
    
    optimizer = Adam(learning_rate=learning_rate, clipnorm=1.0)
    
    # CRITICAL FIX: Stronger focal loss to combat class imbalance
    # gamma=3.0 focuses more on hard-to-classify examples (Low/High yields)
    # alpha=0.3 increased to give more weight to minority classes
    model.compile(
        optimizer=optimizer,
        loss=focal_loss(gamma=3.0, alpha=0.3),  # Increased from gamma=2.0, alpha=0.25
        metrics=['accuracy',
                 tf.keras.metrics.Precision(name='precision'),
                 tf.keras.metrics.Recall(name='recall')]
    )
    
    return model

hybrid_model = build_hybrid_cnn_gru_model(
    sequence_length=sequence_length,
    n_temporal_features=X_hybrid_temp_aug.shape[2],
    n_static_features=X_hybrid_stat_aug.shape[1],
    learning_rate=0.0002  # Reduced from 0.0003 for more stable training
)
hybrid_model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ temporal_input      │ (None, 12, 17)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 12, 64)    │      3,328 │ temporal_input[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 12, 64)    │        256 │ conv1d_3[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_2     │ (None, 6, 64)     │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_11          │ (None, 6, 64)     │          0 │ max_pooling1d_2[… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 6, 128)    │     24,704 │ dropout_11[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 6, 128)    │        512 │ conv1d_4[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ static_input        │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_12          │ (None, 6, 128)    │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 128)       │      2,048 │ static_input[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, 6, 256)    │     98,560 │ dropout_12[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_7[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 6, 256)    │      1,024 │ conv1d_5[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_15          │ (None, 128)       │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_13          │ (None, 6, 256)    │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 96)        │     12,384 │ dropout_15[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_3     │ (None, 6, 192)    │    203,904 │ dropout_13[0][0]  │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 96)        │        384 │ dense_8[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 534,691 (2.04 MB)

 Trainable params: 531,907 (2.03 MB)

 Non-trainable params: 2,784 (10.88 KB)

In [24]:
# Train Hybrid model with ANTI-OVERFITTING strategy
print("\n🚀 Training IMPROVED Hybrid CNN-GRU model...")
print("ANTI-OVERFITTING IMPROVEMENTS:")
print("  • INCREASED L2 regularization (0.001 → 0.01) to prevent memorization")
print("  • INCREASED dropout (0.2-0.35 → 0.3-0.45) for better generalization")
print("  • Added residual connections for gradient flow")
print("  • Stronger focal loss (gamma=3.0) for hard examples")
print("  • Lower learning rate (0.0002) for stability")
print("  • Enhanced class weights for Low/High yield detection")

hybrid_callbacks = [
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=40,  # Increased patience for slower learning rate
        restore_best_weights=True,
        verbose=1
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.6,  # Less aggressive reduction
        patience=15,  # More patience
        min_lr=1e-7,
        verbose=1
    ),
    callbacks.ModelCheckpoint(
        filepath='models/hybrid_best.weights.h5',
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=True,
        verbose=1
    )
]

# CRITICAL FIX: Amplify class weights for better Low/High yield detection
# The original weights were too balanced, causing Medium bias
hybrid_class_weight_amplified = {}
for cls, weight in hybrid_class_weight_dict.items():
    if cls == 0 or cls == 2:  # Low and High classes
        hybrid_class_weight_amplified[cls] = weight * 1.5  # 50% boost
    else:  # Medium class
        hybrid_class_weight_amplified[cls] = weight * 0.7  # Reduce medium weight

print("\nAmplified Class Weights (to combat Medium bias):")
for cls, weight in hybrid_class_weight_amplified.items():
    class_names = ['Low', 'Medium', 'High']
    print(f"  Class {cls} ({class_names[cls]}): {weight:.4f}")

history_hybrid = hybrid_model.fit(
    [X_hybrid_temp_aug, X_hybrid_stat_aug],
    y_hybrid_train_aug,
    validation_data=([X_hybrid_temp_val_scaled, X_hybrid_stat_val_scaled], y_hybrid_val_onehot),
    epochs=300,  # Keep high epochs due to early stopping
    batch_size=24,  # Smaller batch for better gradient estimation
    class_weight=hybrid_class_weight_amplified,  # Use amplified weights
    callbacks=hybrid_callbacks,
    verbose=1
)

print("\n✓ Hybrid model training complete with improvements!")


🚀 Training IMPROVED Hybrid CNN-GRU model...
ANTI-OVERFITTING IMPROVEMENTS:
  • INCREASED L2 regularization (0.001 → 0.01) to prevent memorization
  • INCREASED dropout (0.2-0.35 → 0.3-0.45) for better generalization
  • Added residual connections for gradient flow
  • Stronger focal loss (gamma=3.0) for hard examples
  • Lower learning rate (0.0002) for stability
  • Enhanced class weights for Low/High yield detection

Amplified Class Weights (to combat Medium bias):
  Class 0 (Low): 1.5000
  Class 1 (Medium): 0.7000
  Class 2 (High): 1.5000
Epoch 1/300
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.3082 - loss: 20.0945 - precision: 0.3105 - recall: 0.2606
Epoch 1: val_loss improved from None to 18.60689, saving model to models/hybrid_best.weights.h5

Epoch 1: finished saving model to models/hybrid_best.weights.h5
54/54 ━━━━━━━━━━━━━━━━━━━━ 30s 108ms/step - accuracy: 0.3542 - loss: 19.6818 - precision: 0.3550 - recall: 0.2986 - val_accuracy: 0.3333 - val_loss: 18.6069 - val_pre

In [25]:
# Evaluate Hybrid model
print("\n📊 Evaluating Hybrid model...")

y_hybrid_pred_train_probs = hybrid_model.predict([X_hybrid_temp_train_scaled, X_hybrid_stat_train_scaled])
y_hybrid_pred_val_probs = hybrid_model.predict([X_hybrid_temp_val_scaled, X_hybrid_stat_val_scaled])
y_hybrid_pred_test_probs = hybrid_model.predict([X_hybrid_temp_test_scaled, X_hybrid_stat_test_scaled])

y_hybrid_pred_train = np.argmax(y_hybrid_pred_train_probs, axis=1)
y_hybrid_pred_val = np.argmax(y_hybrid_pred_val_probs, axis=1)
y_hybrid_pred_test = np.argmax(y_hybrid_pred_test_probs, axis=1)

print("\n" + "="*80)
print("HYBRID CNN-GRU MODEL RESULTS")
print("="*80)

for split_name, y_true, y_pred in [
    ('Train', y_hybrid_train_cat, y_hybrid_pred_train),
    ('Validation', y_hybrid_val_cat, y_hybrid_pred_val),
    ('Test', y_hybrid_test_cat, y_hybrid_pred_test)
]:
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    
    print(f"\n{split_name} Set:")
    print(f"  Accuracy:  {acc:.4f}")
    print(f"  Precision: {prec:.4f}")
    print(f"  Recall:    {rec:.4f}")
    print(f"  F1-Score:  {f1:.4f}")

print("\n\nTest Set Confusion Matrix:")
print(confusion_matrix(y_hybrid_test_cat, y_hybrid_pred_test))

print("\n\nTest Set Classification Report:")
print(classification_report(y_hybrid_test_cat, y_hybrid_pred_test,
                          target_names=['Low', 'Medium', 'High']))


📊 Evaluating Hybrid model...
11/11 ━━━━━━━━━━━━━━━━━━━━ 10s 358ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step

HYBRID CNN-GRU MODEL RESULTS

Train Set:
  Accuracy:  1.0000
  Precision: 1.0000
  Recall:    1.0000
  F1-Score:  1.0000

Validation Set:
  Accuracy:  0.7407
  Precision: 0.8000
  Recall:    0.7407
  F1-Score:  0.7083

Test Set:
  Accuracy:  0.9074
  Precision: 0.9082
  Recall:    0.9074
  F1-Score:  0.9073


Test Set Confusion Matrix:
[[18  0  0]
 [ 0 15  3]
 [ 0  2 16]]


Test Set Classification Report:
              precision    recall  f1-score   support

         Low       1.00      1.00      1.00        18
      Medium       0.88      0.83      0.86        18
        High       0.84      0.89      0.86        18

    accuracy                           0.91        54
   macro avg       0.91      0.91      0.91        54
weighted avg       0.91      0.91      0.91        54



---
## Model Comparison

In [26]:
# Compare all models
print("\n" + "="*80)
print("FINAL MODEL COMPARISON (Test Set)")
print("="*80)

comparison_data = {
    'Model': ['CNN', 'GRU', 'Hybrid CNN-GRU'],
    'Accuracy': [
        accuracy_score(y_cnn_test_cat, y_cnn_pred_test),
        accuracy_score(y_gru_test_cat, y_gru_pred_test),
        accuracy_score(y_hybrid_test_cat, y_hybrid_pred_test)
    ],
    'Precision': [
        precision_score(y_cnn_test_cat, y_cnn_pred_test, average='weighted', zero_division=0),
        precision_score(y_gru_test_cat, y_gru_pred_test, average='weighted', zero_division=0),
        precision_score(y_hybrid_test_cat, y_hybrid_pred_test, average='weighted', zero_division=0)
    ],
    'Recall': [
        recall_score(y_cnn_test_cat, y_cnn_pred_test, average='weighted', zero_division=0),
        recall_score(y_gru_test_cat, y_gru_pred_test, average='weighted', zero_division=0),
        recall_score(y_hybrid_test_cat, y_hybrid_pred_test, average='weighted', zero_division=0)
    ],
    'F1-Score': [
        f1_score(y_cnn_test_cat, y_cnn_pred_test, average='weighted', zero_division=0),
        f1_score(y_gru_test_cat, y_gru_pred_test, average='weighted', zero_division=0),
        f1_score(y_hybrid_test_cat, y_hybrid_pred_test, average='weighted', zero_division=0)
    ]
}

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.round(4)

print("\n")
print(comparison_df.to_string(index=False))

print("\n\nBest Model by Metric:")
for metric in ['Accuracy', 'Precision', 'Recall', 'F1-Score']:
    best_idx = comparison_df[metric].idxmax()
    best_model = comparison_df.loc[best_idx, 'Model']
    best_score = comparison_df.loc[best_idx, metric]
    print(f"  {metric}: {best_model} ({best_score:.4f})")


FINAL MODEL COMPARISON (Test Set)


         Model  Accuracy  Precision  Recall  F1-Score
           CNN    0.7407     0.7831  0.7407    0.7285
           GRU    0.6852     0.8381  0.6852    0.5948
Hybrid CNN-GRU    0.9074     0.9082  0.9074    0.9073


Best Model by Metric:
  Accuracy: Hybrid CNN-GRU (0.9074)
  Precision: Hybrid CNN-GRU (0.9082)
  Recall: Hybrid CNN-GRU (0.9074)
  F1-Score: Hybrid CNN-GRU (0.9073)


---
## Save Models

In [27]:
# Save complete models
print("\n💾 Saving models...")

cnn_model.save('models/cnn_model.keras')
gru_model.save('models/gru_model.keras')
hybrid_model.save('models/hybrid_model.keras')

print("✓ Models saved:")
print("  - models/cnn_model.keras")
print("  - models/gru_model.keras")
print("  - models/hybrid_model.keras")
print("  - models/cnn_best.weights.h5")
print("  - models/gru_best.weights.h5")
print("  - models/hybrid_best.weights.h5")

# Save scalers and encoders
import joblib

print("\n💾 Saving scalers...")
joblib.dump(scaler_cnn, 'models/cnn_scaler.pkl')
joblib.dump(scaler_gru, 'models/gru_scaler.pkl')
joblib.dump(scaler_hybrid_temp, 'models/hybrid_temp_scaler.pkl')
joblib.dump(scaler_hybrid_stat, 'models/hybrid_stat_scaler.pkl')

print("✓ Scalers saved:")
print("  - models/cnn_scaler.pkl")
print("  - models/gru_scaler.pkl")
print("  - models/hybrid_temp_scaler.pkl")
print("  - models/hybrid_stat_scaler.pkl")

print("\n💾 Saving encoders...")
joblib.dump(crop_encoder, 'models/crop_encoder.pkl')
joblib.dump(region_encoder, 'models/region_encoder.pkl')

print("✓ Encoders saved:")
print("  - models/crop_encoder.pkl")
print("  - models/region_encoder.pkl")

print("\n✅ All models, scalers, and encoders saved successfully!")


💾 Saving models...
✓ Models saved:
  - models/cnn_model.keras
  - models/gru_model.keras
  - models/hybrid_model.keras
  - models/cnn_best.weights.h5
  - models/gru_best.weights.h5
  - models/hybrid_best.weights.h5

💾 Saving scalers...
✓ Scalers saved:
  - models/cnn_scaler.pkl
  - models/gru_scaler.pkl
  - models/hybrid_temp_scaler.pkl
  - models/hybrid_stat_scaler.pkl

💾 Saving encoders...
✓ Encoders saved:
  - models/crop_encoder.pkl
  - models/region_encoder.pkl

✅ All models, scalers, and encoders saved successfully!


In [28]:
# VERIFY AND SAVE MODELS
from pathlib import Path
import joblib
import os

print("="*80)
print("SAVING TRAINED MODELS")
print("="*80)

# Ensure models directory exists
Path('models').mkdir(exist_ok=True)
print("\n✓ Models directory created/verified")

# Check models are in memory
print("\n📊 Models in memory:")
print(f"  CNN model: {type(cnn_model).__name__}")
print(f"  GRU model: {type(gru_model).__name__}")
print(f"  Hybrid model: {type(hybrid_model).__name__}")

print("\n💾 Saving models...")
cnn_model.save('models/cnn_model.keras')
print("  ✓ models/cnn_model.keras")

gru_model.save('models/gru_model.keras')
print("  ✓ models/gru_model.keras")

hybrid_model.save('models/hybrid_model.keras')
print("  ✓ models/hybrid_model.keras")

print("\n💾 Saving scalers...")
joblib.dump(scaler_cnn, 'models/cnn_scaler.pkl')
joblib.dump(scaler_gru, 'models/gru_scaler.pkl')
joblib.dump(scaler_hybrid_temp, 'models/hybrid_temp_scaler.pkl')
joblib.dump(scaler_hybrid_stat, 'models/hybrid_stat_scaler.pkl')
print("  ✓ All 4 scalers saved")

print("\n💾 Saving encoders...")
joblib.dump(crop_encoder, 'models/crop_encoder.pkl')
joblib.dump(region_encoder, 'models/region_encoder.pkl')
print("  ✓ All 2 encoders saved")

print("\n📁 Verifying files...")
files = sorted(os.listdir('models'))
for f in files:
    size = os.path.getsize(f'models/{f}') / (1024*1024)
    print(f"  {f:<35} {size:>8.2f} MB")

print("\n" + "="*80)
print("✅ ALL MODELS, SCALERS, AND ENCODERS SAVED SUCCESSFULLY!")
print("="*80)

SAVING TRAINED MODELS

✓ Models directory created/verified

📊 Models in memory:
  CNN model: Sequential
  GRU model: Sequential
  Hybrid model: Functional

💾 Saving models...
  ✓ models/cnn_model.keras
  ✓ models/gru_model.keras
  ✓ models/hybrid_model.keras

💾 Saving scalers...
  ✓ All 4 scalers saved

💾 Saving encoders...
  ✓ All 2 encoders saved

📁 Verifying files...
  cnn_best.weights.h5                     2.07 MB
  cnn_model.keras                         2.08 MB
  cnn_scaler.pkl                          0.00 MB
  crop_encoder.pkl                        0.00 MB
  gru_best.weights.h5                     2.50 MB
  gru_model.keras                         2.52 MB
  gru_scaler.pkl                          0.00 MB
  hybrid_best.weights.h5                  6.26 MB
  hybrid_model.keras                      6.30 MB
  hybrid_stat_scaler.pkl                  0.00 MB
  hybrid_temp_scaler.pkl                  0.00 MB
  region_encoder.pkl                      0.00 MB

✅ ALL MODELS, SCALERS, AND

---
## Summary

✅ **Three models trained successfully:**
- **CNN**: 1D convolutions for temporal pattern extraction
- **GRU**: Bidirectional recurrent network for sequence modeling
- **Hybrid CNN-GRU**: Combined architecture with CNN→GRU pipeline + static features

✅ **Improvements implemented:**
- Lag features (previous 1-3 years' yields)
- Data augmentation (2x training data)
- Class weights for balanced learning
- Bidirectional processing for temporal context

✅ **Next Steps:**
Proceed to phase4_validation.ipynb for detailed analysis and ensemble modeling.

In [29]:

# SAVE MODELS FROM KERNEL
from pathlib import Path
import joblib

print("="*80)
print("SAVING MODELS")
print("="*80)

Path('models').mkdir(exist_ok=True)

print("\n💾 Saving CNN model...")
cnn_model.save('models/cnn_model.keras')
print("  ✓ models/cnn_model.keras")

print("\n💾 Saving GRU model...")
gru_model.save('models/gru_model.keras')
print("  ✓ models/gru_model.keras")

print("\n💾 Saving Hybrid model...")
hybrid_model.save('models/hybrid_model.keras')
print("  ✓ models/hybrid_model.keras")

print("\n💾 Saving scalers...")
joblib.dump(scaler_cnn, 'models/cnn_scaler.pkl')
joblib.dump(scaler_gru, 'models/gru_scaler.pkl')
joblib.dump(scaler_hybrid_temp, 'models/hybrid_temp_scaler.pkl')
joblib.dump(scaler_hybrid_stat, 'models/hybrid_stat_scaler.pkl')
print("  ✓ All scalers saved")

print("\n💾 Saving encoders...")
joblib.dump(crop_encoder, 'models/crop_encoder.pkl')
joblib.dump(region_encoder, 'models/region_encoder.pkl')
print("  ✓ All encoders saved")

import os
print("\n📁 Files created:")
for f in sorted(os.listdir('models')):
    size = os.path.getsize(f'models/{f}') / (1024*1024)
    print(f"  {f:<35} {size:>8.2f} MB")

print("\n" + "="*80)
print("✅ ALL FILES SAVED!")
print("="*80)


SAVING MODELS

💾 Saving CNN model...
  ✓ models/cnn_model.keras

💾 Saving GRU model...
  ✓ models/gru_model.keras

💾 Saving Hybrid model...
  ✓ models/hybrid_model.keras

💾 Saving scalers...
  ✓ All scalers saved

💾 Saving encoders...
  ✓ All encoders saved

📁 Files created:
  cnn_best.weights.h5                     2.07 MB
  cnn_model.keras                         2.08 MB
  cnn_scaler.pkl                          0.00 MB
  crop_encoder.pkl                        0.00 MB
  gru_best.weights.h5                     2.50 MB
  gru_model.keras                         2.52 MB
  gru_scaler.pkl                          0.00 MB
  hybrid_best.weights.h5                  6.26 MB
  hybrid_model.keras                      6.30 MB
  hybrid_stat_scaler.pkl                  0.00 MB
  hybrid_temp_scaler.pkl                  0.00 MB
  region_encoder.pkl                      0.00 MB

✅ ALL FILES SAVED!
